# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846792912941                   -0.70    4.0
  2   -7.852298406492       -2.26       -1.53    1.0
  3   -7.852646013689       -3.46       -2.52    3.2
  4   -7.852646677217       -6.18       -3.33    2.5
  5   -7.852646685912       -8.06       -4.80    1.2
  6   -7.852646686726       -9.09       -5.23    4.2
  7   -7.852646686730      -11.42       -5.99    2.0
  8   -7.852646686730      -12.74       -7.62    2.0
  9   -7.852646686730      -14.75       -8.02    2.5
 10   -7.852646686730   +  -14.75       -9.14    1.5
 11   -7.852646686730      -15.05       -9.93    3.2
 12   -7.852646686730      -14.57       -9.30    1.8
 13   -7.852646686730   +  -14.45       -7.83    4.0
 14   -7.852646686730      -14.45       -7.88    2.2
 15   -7.852646686730   +  -14.57       -7.84    2.8
 16   -7.852646686730   +  -15.05       -7.39    4.8
 17   -7.852646686730      -14.57       -8.04 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846876154313                   -0.70           4.0
  2   -7.852524937813       -2.25       -1.62   0.80    4.8
  3   -7.852609957687       -4.07       -2.79   0.80    5.2
  4   -7.852646570442       -4.44       -3.55   0.80    7.2
  5   -7.852646683644       -6.95       -4.56   0.80    8.5
  6   -7.852646686690       -8.52       -5.15   0.80   10.5
  7   -7.852646686726      -10.45       -6.38   0.80   11.8
  8   -7.852646686730      -11.37       -7.04   0.80   13.8
  9   -7.852646686730      -13.75       -7.48   0.80   15.0
 10   -7.852646686730   +    -Inf       -8.17   0.80   15.5
 11   -7.852646686730   +  -15.05       -9.22   0.80   17.0
 12   -7.852646686730   +  -15.05      -10.24   0.80   18.8
 13   -7.852646686730      -15.05      -11.24   0.80   20.8
 14   -7.852646686730      -14.57      -11.67   0.80   22.2
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.385342e+01     5.517291e+00
 * time: 0.5543420314788818
     1     1.120743e+00     2.884817e+00
 * time: 0.8125600814819336
     2    -1.116136e+00     3.327548e+00
 * time: 0.84151291847229
     3    -3.444695e+00     3.079112e+00
 * time: 0.8930609226226807
     4    -4.706742e+00     2.760815e+00
 * time: 0.933603048324585
     5    -6.441983e+00     1.564770e+00
 * time: 0.9749279022216797
     6    -7.142300e+00     5.595088e-01
 * time: 1.0159258842468262
     7    -7.443254e+00     2.943132e-01
 * time: 1.044753074645996
     8    -7.606423e+00     1.821207e-01
 * time: 1.074207067489624
     9    -7.729573e+00     2.046729e-01
 * time: 1.1032719612121582
    10    -7.790899e+00     1.242413e-01
 * time: 1.132802963256836
    11    -7.831209e+00     7.220831e-02
 * time: 1.1613759994506836
    12    -7.843947e+00     6.704512e-02
 * time: 1.190100908279419
    13    -7.849790e+00     2.552547e-02
 * time: 1.2189810276031494


## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846727527190                   -0.70    4.0
  2   -7.852314664474       -2.25       -1.54    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.54
  2   -7.852646686730      -10.67       -5.87
  3   -7.852646686730      -15.05      -12.53


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 9.6597483561996e-14
|ρ_newton - ρ_scfv| = 1.4323200635833255e-13
|ρ_newton - ρ_dm|   = 7.976347441593329e-10
